In [ ]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration

dataset = load_dataset('IlyaGusev/gazeta')


In [ ]:
dataset['train'][0]

{'text': 'Сегодня транспортный налог начисляется в зависимости от мощности автомобиля, причем цена для «сильных» машин выше, чем для малолитражек. Также ставку налога могут корректировать региональные власти: согласно Налоговому кодексу, базовый тариф, установленный правительством, может быть уменьшен в пять раз или увеличен до 10 раз. Сборы идут в региональные бюджеты, откуда растекаются на общие нужды. Транспортный налог — один из основных источников бюджетных доходов — предлагается направить исключительно на дорожные фонды. Так, автомобилисты будут понимать, за что они платят, а дорожники будут иметь гарантированный доход. Кроме налога дорожные фонды будут пополняться за счет бюджетных средств и проезда по платным дорогам. Более того, транспортный налог предлагается завуалировать в акцизы на бензин. Привычную и раздражающую систему ежегодной оплаты квитанции предлагается изменить, включив налог в стоимость топлива. Минэкономразвития говорит об удвоении акцизы, которая сегодня состав

In [ ]:
import torch

lr=3e-4
max_input_length = 768
max_target_length = 60
batch_size = 4
epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = T5Tokenizer.from_pretrained('cointegrated/rut5-base')
model = T5ForConditionalGeneration.from_pretrained('cointegrated/rut5-base').to(device)


def preprocessing(examples):
    inputs = ['summarize: ' + doc for doc in examples['text']]

    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding=False,
    )

    labels = tokenizer(
        examples["summary"],
        max_length=max_target_length,
        truncation=True,
        padding=False,
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_dataset = dataset.map(
    preprocessing,
    batched=True
)

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00


In [ ]:
import evaluate
import numpy as np
from transformers import Trainer, TrainingArguments
from tqdm.auto import tqdm


def compute_metrics(eval_preds, tokenizer):
    rouge = evaluate.load("rouge")

    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=["rouge1", "rouge2", "rougeL"]
    )

    prediction_lens = [len(pred.split()) for pred in decoded_preds]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
training_args = TrainingArguments(
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size * 2,
    learning_rate=lr,
    weight_decay=0.01,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=torch.cuda.is_available(),
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=lambda p: compute_metrics(p, tokenizer),
)

trainer.train()
print("TRAINED")

In [ ]:
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import time


def collate_fn(batch):
    """Собирает батч и делает динамический паддинг"""
    input_ids = [item["input_ids"] for item in batch]
    attention_mask = [item["attention_mask"] for item in batch]
    labels = [item["labels"] for item in batch]

    # Паддинг входов
    input_ids = torch.nn.utils.rnn.pad_sequence(
        [torch.tensor(ids) for ids in input_ids],
        batch_first=True,
        padding_value=tokenizer.pad_token_id
    )

    attention_mask = torch.nn.utils.rnn.pad_sequence(
        [torch.tensor(mask) for mask in attention_mask],
        batch_first=True,
        padding_value=0
    )

    # Паддинг лейблов (с -100 для игнорирования паддинга в loss)
    labels = torch.nn.utils.rnn.pad_sequence(
        [torch.tensor(lab) for lab in labels],
        batch_first=True,
        padding_value=-100
    )

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

train_loader = DataLoader(
    tokenized_dataset["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=2,
    pin_memory=True if device.type == "cuda" else False,
)

val_loader = DataLoader(
    tokenized_dataset["validation"],
    batch_size=batch_size * 2,  # На валидации можно больший батч
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=2,
    pin_memory=True if device.type == "cuda" else False,
)


def train_epoch(model, train_loader, criterion, opt, epoch, device):
    epoch_loss = []

    model.train()
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch}')

    for batch_idx, batch in enumerate(progress_bar):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"]
        )

        loss = outputs.loss

        loss.backward()
        opt.step()
        opt.zero_grad()

        epoch_loss.append(loss.item())

    print(f'EPOCH {i + 1}')
    print(f'TRAINING LOSS {np.mean(epoch_loss)}')
    return np.mean(epoch_loss)


def evaluate(model, val_loader, criterion, device):
    epoch_loss = []

    model.eval()
    progress_bar = tqdm(val_loader, desc='Validation')
    with torch.no_grad():
        for batch in progress_bar:
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                labels=batch["labels"]
            )

            epoch_loss.append(outputs.loss.item())
    print('VAL LOSS', np.mean(epoch_loss))
    return np.mean(epoch_loss)


model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters())
for i in range(epochs):
    start_time = time.time()
    train_epoch(model, train_loader, criterion, opt, i, device)
    evaluate(model)
    end_time = time.time()

In [ ]:
def test_model(model, tokenizer, text: str, num_beams: int = 4):
    input_text = "summarize: " + text

    inputs = tokenizer(
        input_text,
        max_length=max_input_length,
        truncation=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_target_length,
        )

    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary


for i in range(5):
    print(f'TEST FOR NUMBER {i + 1}')
    print(dataset['test'][i]['text'])
    print(test_model(dataset['test'][i]['text']))
    print()
    print('PROJECTED SUMMARY:')
    print(dataset['test'][i]['summary'])

